In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import keras


from keras.initializations import he_normal

from keras.models import Model
from keras.regularizers import l2
from keras.layers import Input, Conv2D,merge
import importlib

if __name__ == "__main__":
    sys.path.append("../../../")
from notebooks_src.models.configs import configs
from notebooks_src.models.util import make_model_data_struct
from notebooks_src.models.base.get_base_model import get_base_model_layers  
from notebooks_src.box_encode_decode.configs import configs as box_configs
configs.update(box_configs)

Using TensorFlow backend.


importing Jupyter notebook from ../../../notebooks_src/models/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/config_util.ipynb
importing Jupyter notebook from ../../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../../notebooks_src/models/util.ipynb
importing Jupyter notebook from ../../../notebooks_src/models/base/get_base_model.ipynb
importing Jupyter notebook from ../../../notebooks_src/box_encode_decode/configs.ipynb


In [2]:
conv_kwargs =  dict(border_mode="same", init=configs["w_init"])#, W_regularizer=l2(configs["w_decay"]))

In [ ]:
def add_ssd_additional_feat_layers():
    #look up in SSD or DSSD paper
    

In [8]:
def get_ssd_detection_outputs(layers):
    lrf_layers_dict, layer_fmap_size_dict = get_base_model_layers()
    num_anchors_dict = make_fmap_size_to_num_anchors_map()
    net_out_dict={}
    num_classes = configs["num_classes"]
    xywh_outputs = []
    cls_outputs = []
    
    for fmap_size, fmap in layer_fmap_size_dict.iteritems():
        xywh_output, cls_output = get_detections_for_fmap(fmap_size, fmap)
        xywh_outputs.append(xywh_output)
        cls_outputs.append(cls_output)
        

        

    return xywh_outputs, cls_outputs

In [5]:
def get_detections_for_fmap(fmap_size, fmap):
        num_anchors = num_anchors_dict[fmap_size]
        num_xywh_outputs = num_anchors * 4
        num_cls_ouputs = num_anchors * num_classes
        xywh_output = Conv2D(fmap, num_xywh_outputs, 3, 3, activation="linear", **conv_kwargs)
        
        #no softmax for now?
        cls_output = Conv2D(fmap, num_cls_ouputs, 3, 3, activation="linear", **conv_kwargs)
        
        return xywh_output, cls_output
        
    

In [6]:
def make_fmap_size_to_num_anchors_map():
    fmap_sizes = configs["feat_shapes"]
    num_boxes = [len(sizes) + len(ratios) for sizes, ratios in zip(configs["anchor_sizes"],configs["anchor_ratios"])]
    return dict(zip(fmap_sizes, num_boxes))

In [7]:
# # layers is a dict matching local receptive field to layer
#pseudoish-code
layers = get_base_model_layers()
layers = add_ssd_additional_feat_layers()
outputs = get_ssd_detection_outputs()

importing Jupyter notebook from ../../../notebooks_src/models/base/resnet50.ipynb
importing Jupyter notebook from ../../../notebooks_src/models/base/resnet_utils.ipynb


In [ ]:
# # encoder is last layer, so layer with largest receptive field
# encoder_layer = layers[max(layers.keys())]

# #input tensor has local recpetive field of 1 (layers is a dict NOT a list)
# input_tensor = layers[1]

# output = bbox(encoder_layer)

In [ ]:
def get_model_params():
    return make_model_data_struct(input=input_tensor, output=output)